# Vector base Faiss Test

In [1]:
!pip install faiss-gpu
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 101.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.7 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=1d76bcbb21a7657bfc6954c5b09345b2205e27a45d6584cd13da824806789a66
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_tra

In [2]:
from sentence_transformers import SentenceTransformer
import json
import faiss


model = SentenceTransformer('BAAI/bge-large-zh-v1.5')

In [3]:
database = json.load(open('/content/drive/MyDrive/Llama2_ch/data/database.json'))

In [4]:
v = list(database.values())

In [5]:
keys = list(database.keys())
keys[-9:]

['什麼是積分？能做什麼？',
 '如何申請成為會員',
 '如何獲得積分？',
 '所獲得的積分，要多久才會顯示於我的帳戶內？',
 '如何兌換獎賞? ',
 '積分會過期嗎?',
 '哪裡可查閱有關積分到期日的詳情？',
 '積分可以分享嗎？',
 '如何更新我的手機號碼?']

In [6]:
k_embeddings = model.encode(keys, normalize_embeddings=True)
k_embeddings.shape

(2136, 1024)

In [8]:
import numpy as np
np.save('/content/drive/MyDrive/Llama2_ch/data/database_embedding', k_embeddings)

In [8]:
dim, measure = k_embeddings.shape[1], faiss.METRIC_L2
param = 'Flat'
index = faiss.index_factory(dim, param, measure)
print(index.is_trained)
if not index.is_trained:
  index.train(k_embeddings)
index.add(k_embeddings)                   # 将向量库中的向量加入到index中
print(index.ntotal)

True
2136


In [11]:
test_sentence = '積分要怎麼拿到'
ts_embedd = model.encode(test_sentence, normalize_embeddings=True)

In [14]:
k = 3                 # topK的K值
D, I = index.search(ts_embedd.reshape(1,-1), k)# xq为待检索向量，返回的I为每个待检索query最相似TopK的索引list，D为其对应的距离
print(D)
for s in [(str(x+1)+'.'+keys[i],v[i]) for x, i in enumerate(I[0])]:
  print('Q:',s[0],'\nA:', s[1])

[[0.3342797  0.57858104 0.6605813 ]]
Q: 1.如何獲得積分？ 
A: 每完成以下一個任務就可以獲得 Kinnso Points： 
a. 每一次成功透過WhatsApp拎著數，即獲得 10 Points!
b, 每一日成功透過WhatsApp 與蜜熊傾計，即獲得 20 Points!
c. 把推薦者連結發送給Kinnso全新用戶朋友，朋友成功透過WhatsApp拎著數後，雙方均可各獲得 30 Points!
d. 不定期推出特別任務，成功完成即獲得 100 Points!
e 優惠報料，優惠一經採用，即獲得 50 points!
Q: 2.積分可以分享嗎？ 
A: 不可以。Kinnso 會籍及積分只限用戶本人使用，不得轉讓他人。
Q: 3.什麼是積分？能做什麼？ 
A: Kinnso Points 是 Kinnso 獎賞計劃，旨在回饋用戶。用戶完成簡單任務及互動，便可獲取額外積分，兌換獎賞。


# Sim test

In [9]:
# 对于短对短数据集，下面是通用的使用方式
import numpy as np
from sentence_transformers import SentenceTransformer

s_model = SentenceTransformer('BAAI/bge-large-zh-v1.5')

In [10]:
history = [['使用者想找美食，請在接下來的對話中，詢問使用者[]內的5個問題，不重複詢問，亂序詢問問題，每輪對話只詢問一個問題，一直到[]內的5個問題都詢問完:[1.美食地區, 2.人數, 3.用餐預算, 4.用餐時段, 5.用餐口味喜好]', '好的，讓我們開始找美食吧！想知道您打算在哪個時刻用餐，您可以選擇不同的用餐時段。'], ['不太確定', '想請問您有特定的美食地區嗎？'], ['信義區吧', '關於用餐預算，您能告訴我大致的數字嗎？'], ['一千左右', '請問有多少人呢？'], ['2人', '請問您有什麼口味喜好或特殊飲食要求嗎？例如，辣的或是不吃香菜，西式或是中式。'], ['板前壽司', None]]

qs = []
ans = []
for i in history:
  if i != history[-1]:
    qs.append(i[1])
  if i != history[0]:
    ans.append(i[0])

keys = ["美食地區", '人數', '用餐預算', '用餐時段', '用餐口味喜好']
embeddings_k = s_model.encode(keys, normalize_embeddings=True)
embeddings_q = s_model.encode(qs, normalize_embeddings=True)
similarity = embeddings_k @ embeddings_q.T

check_exist = []
for numbers,k in zip(similarity, keys):
  ind = np.argsort(numbers)
  if ind[-1] not in check_exist:
    check_exist.append(ind[-1])
    print(k+':'+ans[ind[-1]])
  else:
    print(k+':'+ans[ind[-2]])

美食地區:信義區吧
人數:2人
用餐預算:一千左右
用餐時段:不太確定
用餐口味喜好:板前壽司


In [11]:
data = {}
data['什麼是「Kinnso Points」？'] = 'Kinnso Points 是 Kinnso 獎賞計劃，旨在回饋用戶。用戶完成簡單任務及互動，便可獲取額外積分，兌換獎賞。'
data['如何申請成為會員'] = '用戶於Kinnso網站成功透過WhatsApp領取優惠，即自動登記成為Kinnso會員。'
data['我可在那裡賺取「Kinnso Points」積分？如何獲得積分？'] = '''每完成以下一個任務就可以獲得 Kinnso Points：
a. 每一次成功透過WhatsApp拎著數，即獲得 10 Points!
b, 每一日成功透過WhatsApp 與蜜熊傾計，即獲得 20 Points!
c. 把推薦者連結發送給Kinnso全新用戶朋友，朋友成功透過WhatsApp拎著數後，雙方均可各獲得 30 Points!
d. 不定期推出特別任務，成功完成即獲得 100 Points!
e 優惠報料，優惠一經採用，即獲得 50 points!'''

data['所賺得的積分，要多久才會顯示於我的帳戶內？'] = 'a-d 積分會於成功完成簡單任務當天，增加並顯示於您的帳戶。e 積分會於成功批核後，14個工作天內，增加並顯示於您的帳戶。'
data['如何兌換獎賞? '] = '用戶可以透過WhatsApp 進入 Kinnso聊天室 ，在Kinnso聊天室輸入「Menu」，再輸入「4」以兌換獎賞。用戶進入「獎賞中心」頁面後，請選擇你想兌換的禮品及數量，然後按「兌換」即領獎賞，獎賞會以優惠碼、二維條碼或專屬網址形式派發。'
data['「Kinnso Points」積分會過期嗎?'] = '''Kinnso Points 積分有效日期詳情如下：
第一輪有效日期：每年於1月1日- 6月30日賺取的積分，將於同年12月31日到期，即會員須於同年12月31日或之前必須使用積分換領獎賞，否則積分於下年1月1日起自動失效。
第二輪有效日期：每年於7月1日- 12月31日賺取的積分，有效期為下年6月30日，即會員須於下年6月30日或之前必須使用積分換領獎賞，否則積分將於下年7月1日起自動失效。
'''

data['哪裡可查閱有關積分到期日的詳情？'] = '用戶可以透過WhatsApp 進入 Kinnso聊天室，在WhatsApp 輸入「Menu」，再輸入「3」以查閱積分。'
data['我能與別人分享積分嗎？'] = '不可以。Kinnso 會籍及積分只限用戶本人使用，不得轉讓他人。'
data['如何更新我的手機號碼?'] = '如會員更改手機號碼，必須立即電郵至 general@kinnso.ai 通知Kinnso，使用新的手機號碼重新領取優惠以登記Kinnso 會籍。如發現任何會員使用非由其本人實際擁有之手機號碼登記或使用Kinnso會籍，Kinnso 有權暫停或終止該會員的資格而不作另行通知。'


In [44]:
keys = list(data.keys())
query = '怎麼賺積分'
embeddings_k = s_model.encode(keys, normalize_embeddings=True)
embeddings_q = s_model.encode(query, normalize_embeddings=True)
similarity = embeddings_k @ embeddings_q.T
print('query:',query,'\nQ:',keys[np.argmax(similarity)],'\nA:',data[keys[np.argmax(similarity)]])

query: 怎麼賺積分 
Q: 我能與別人分享積分嗎？ 
A: 不可以。Kinnso 會籍及積分只限用戶本人使用，不得轉讓他人。
